In [28]:
import pandas as pd
from functools import reduce
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

## Loading data

In [31]:
year = "2021" # change this accordingly!

path = "../data/uefa-data/processed/" + year + "/" + year + "_"

playerStats = pd.read_csv(path + "player_stats.csv")
defense = pd.read_csv(path + "defense.csv")
goalkeeping = pd.read_csv(path + "goalkeeping.csv")
passing = pd.read_csv(path + "passing.csv")
shooting = pd.read_csv(path + "shooting.csv")

# merge all data into one by player id
merge = [playerStats, defense, passing, shooting]
playerData = reduce(lambda left, right: pd.merge(left, right, on = ["Player"]), merge)
#playerData = playerData.merge(goalkeeping, how='left', on="Player")
# playerData.to_csv(path + "player_data.csv", index = False)
print(playerData)

       Player  Age  90s        mv  DF  FW  GK  MF  Tack Tkl  Tack TklW  ...  \
0    8bc373ea   24  5.0   6000000   1   0   0   0      1.40       0.20  ...   
1    f586779e   22  2.6  38000000   0   1   0   0      0.00       0.00  ...   
2    b96b595c   32  8.0  10000000   1   0   0   0      1.75       1.00  ...   
3    81442ecb   28  3.6  15000000   1   0   0   0      2.50       2.22  ...   
4    2b09d998   21  3.2  20000000   1   0   0   1      2.19       2.19  ...   
..        ...  ...  ...       ...  ..  ..  ..  ..       ...        ...  ...   
745  028e70b9   19  0.1   8000000   0   0   0   1      0.00       0.00  ...   
746  6622454d   27  3.9  38000000   0   1   0   1      2.31       1.54  ...   
747  ce4246f5   25  5.0  32000000   1   0   0   0      1.40       1.40  ...   
748  adfbe1f5   23  4.3   2000000   0   1   0   1      1.86       1.40  ...   
749  79300479   21  1.5  40000000   0   0   0   1      0.67       0.67  ...   

     Stan G/SoT  Stan Dist  Stan FK  Perf PK  Perf 

## Building model

In [32]:
# make train + test split
X = playerData.drop(columns = ["Player", "mv"])
y = playerData["mv"]

XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2, random_state = 114514)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').